
製作一款資訊應用，大概分成四種階段

設計, 開發, 測試, 部署監控

在商業項目定案之後，

會探討到製作面的事情，User Story / Activity Diag/ ER-Model

透過 User Story，我們會開始思考如何製作，用戶使用的視覺化介面

關於Line Chatbot的視覺化設計，已經在設計篇章提過了

通常設計師畫出來的圖，還需要再經過一系列工程化的方式來修繕

就像等等我們要做的圖文選單

稍後的流程會是

圖文選單照片的準備

圖文選單照片尺寸的設定

圖文選單設定檔的準備

把圖文選單圖片與設定檔上傳到Colab

透過程式碼，把圖文選單上傳給Line

取回圖文選單的id

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install line-bot-sdk

     |████████████████████████████████| 71kB 2.2MB/s 


In [4]:
'''
與Google做驗證，取得權限
'''

# 引用套件
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 驗證準備
gauth = GoogleAuth()

# 載入本地端的設定檔
gauth.LoadCredentialsFile("mycreds.txt")

# 若裡面沒有設定檔，則用Colab常使用的驗證方式取得設定檔
if gauth.credentials is None:
  auth.authenticate_user()   
  gauth.credentials = GoogleCredentials.get_application_default() 
elif gauth.access_token_expired:
  gauth.Refresh()  
else:
  gauth.Authorize()

# 將新生成的設定檔儲存起來
gauth.SaveCredentialsFile("mycreds.txt")

drive = GoogleDrive(gauth)

/usr/local/lib/python3.6/dist-packages/oauth2client/_helpers.py:255: UserWarning: Cannot access mycreds.txt: No such file or directory
  warnings.warn(_MISSING_FILE_MESSAGE.format(filename))


In [7]:
'''
迴圈讀取本地列表，
    上傳設定檔，取得id，並將id寫入檔案中，而後上傳圖片

'''

import json
from linebot import LineBotApi

# 載入安全設定檔
line_bot_api = LineBotApi("Uz5zPLpo23iSRE4MnH1s4tyPuQiJE7gEzgZoLH+v/dF7Lq+L3mX/R2ZdV9nTlzuR/YMKEnRpEG7Ep0iYFflWiY5OheAY+sEt1a3IMw11mxP7SbWLqNNnVGRN2DWhhjydOjMtFFe6h13SPPnbxs/zOgdB04t89/1O/w1cDnyilFU=")

'''
rich_menu的本地列表
'''

# 指定material資料夾內的資料夾名
rich_menu_array=['rich_menu_0']

from linebot.models import RichMenu

for rich_menu_name in rich_menu_array:

    # 讀取 rich-menu的json
    root_folder = drive.ListFile({'q': "title='material' "}).GetList()

    richmenu_image_content=''
    file_content=''
    for root_object in root_folder:
      if root_object['mimeType'] == 'application/vnd.google-apps.folder':
        
        # print(anything)
        material_folder = drive.ListFile({'q': " '{}' in parents and trashed=false".format(root_object['id'])}).GetList()
        # print(fileList2)
        for anything_in_material_folder in material_folder:
          # print(replyJson)
          if anything_in_material_folder['title'] == rich_menu_name:
            # print(anything_in_reply_folder)
            target_folder = drive.ListFile({'q': " '{}' in parents and trashed=false".format(anything_in_material_folder['id'])}).GetList()
            for reply_json in target_folder:
              # print(target)
              if reply_json['title'] == 'rich_menu.json':
                file_content=reply_json.GetContentString()
              elif reply_json['title'] == 'rich_menu.jpg':
                reply_json.GetContentFile('temp.jpg')
    rich_menu_json = json.loads(file_content)


    # 創建菜單，取得menuId
    lineRichMenuId = line_bot_api.create_rich_menu(rich_menu=RichMenu.new_from_json_dict(rich_menu_json))
    print("-設定檔上傳結果")
    print(lineRichMenuId)

    # 上傳照片至該menu-id
    set_image_response=''
    with open("temp.jpg", 'rb') as f:
        set_image_response=line_bot_api.set_rich_menu_image(lineRichMenuId, 'image/jpeg', f)


    # id寫入Google指定資料夾端                              
    root_folder = drive.ListFile({'q': "title='material' "}).GetList()

    for root_object in root_folder:
      if root_object['mimeType'] == 'application/vnd.google-apps.folder':
        # print(anything)
        material_folder = drive.ListFile({'q': " '{}' in parents and trashed=false".format(root_object['id'])}).GetList()
        for anything_in_material_folder in material_folder:
          # print(replyJson)
          if anything_in_material_folder['title']== rich_menu_name :
            target_folder = drive.ListFile({'q': " '{}' in parents and trashed=false".format(anything_in_material_folder['id'])}).GetList()
            
            target_file = drive.CreateFile({'title': 'rich_menu_id','parents': [{'id': anything_in_material_folder['id']}]})

            target_file.SetContentString(lineRichMenuId)
            target_file.Upload()                                   
        
    print("-圖片上傳結果")                                               
    print(set_image_response)

-設定檔上傳結果
richmenu-9cc9d8ec74e2aa6662c48acecb96ea0a
-圖片上傳結果
None


In [ ]:
'''

查詢帳號內擁有的richmenu 

'''

from linebot import (
    LineBotApi
)

import json

line_bot_api = LineBotApi("channel_access_token")

line_bot_api.get_rich_menu_list()

In [ ]:
'''

移除帳號內的richmenu

'''

from linebot import (
    LineBotApi
)


line_bot_api = LineBotApi("channel_access_token")

# 設定要移除的rich_menu
rich_menu_name_array = ["rich_menu_0"]

for rich_menu_name in rich_menu_name_array:
    
    # 讀取rich_menu_id檔案，並告知 Line 進行刪除，並在刪除後，把本地檔案內容清除
    with open("素材/"+rich_menu_name+'/rich_menu_id', 'r') as myfile:
        rich_menu_id = myfile.read()
        deleteResult = line_bot_api.delete_rich_menu(rich_menu_id)
        print(deleteResult)
        
    f = open("素材/"+rich_menu_name+"/rich_menu_id", "w")
    f.write('')
    f.close()

In [ ]:
'''

解除用戶綁定

'''

from linebot import (
    LineBotApi
)

import json

# rich_menu_id_array = ["rich_menu_0"]

line_bot_api = LineBotApi("channel_access_token")

self_user_id='Your_User_id'
line_bot_api.unlink_rich_menu_from_user(self_user_id)

In [ ]:
'''

綁定個人用戶進行測試

'''

from linebot import (
    LineBotApi
)

import json

# rich_menu_id_array = ["rich_menu_0"]
line_bot_api = LineBotApi("channel_access_token")

self_user_id='Ud29c5cf085e845db7c6457677116df4d'
rich_menu_id='richmenu-62d1b8717117640cdb9910e273452d35'

line_bot_api.unlink_rich_menu_from_user(self_user_id)
line_bot_api.link_rich_menu_to_user(self_user_id,rich_menu_id)